In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from pyvis.network import Network
from concurrent.futures import ThreadPoolExecutor, as_completed
import networkx as nx
import logging
import os
import glob
from tqdm.notebook import tqdm
import pymaid
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:


catmaid_url = 'https://l1em.catmaid.virtualflybrain.org'
http_user = None
http_password = None
project_id = 1

rm = pymaid.CatmaidInstance(catmaid_url, http_user, http_password, project_id)

# Neural structures
Load and build the structure of particular neurons

In [3]:
class simplified_structure:
    def __init__(self, neuron:pymaid.CatmaidNeuronList, silence=False):
        self.silence = silence
        if silence:
            logging.getLogger('pymaid').setLevel(logging.ERROR)
        self.neuron:pymaid.CatmaidNeuronList = neuron
        self.nodes:nx.MultiDiGraph = None
        # названия в формате id_обекта, если знак id положителен, то это нода скелета, если отрицателен то это коннектор
        self.build_structure()
        self.add_connectors_to_graph()
        self.simplify_directed_graph()

    def build_structure(self):
        nodes = self.neuron.nodes
        graph = nx.MultiDiGraph()
        for idt, parent, ntype in zip(nodes['node_id'], nodes['parent_id'], nodes['type']):
            # сома это root
            if (a:=idt >= 0):
                graph.add_node(idt, type = ntype) 
            if (b:=parent >= 0):
                graph.add_node(parent, type = ntype)
            if a and b:
                graph.add_edge(idt, parent, nodes_inside = [])
        self.nodes = graph

    def add_connectors_to_graph(self):
        connectors = pymaid.get_connectors(self.neuron)
        for idt, ctype in zip(connectors['connector_id'], connectors['type']):
            cID = -idt
            q = self.neuron.connectors[self.neuron.connectors['connector_id'] == idt]
            self.nodes.add_node(cID, type = ctype)
            for node_id in q['node_id']:
                if ctype == 'Postsynaptic':
                    self.nodes.add_edge(cID, node_id, nodes_inside = [])
                elif ctype == 'Presynaptic':
                    self.nodes.add_edge(node_id, cID, nodes_inside = [])
                else:
                    if not self.silence:
                        print(f"finded connector {idt} of type {ctype}")
                    self.nodes.add_edge(node_id, cID, nodes_inside = [])
                    self.nodes.add_edge(cID, node_id, nodes_inside = [])

    def simplify_directed_graph(self):
        def keep_nodes(graph, vid):
            return graph.nodes(True)[vid]['type'] == 'root' or vid < 0
        G = self.nodes
        original = len(self.nodes)
        while True:
            # Находим все вершины с in-degree=1 и out-degree=1
            nodes_to_remove = [
                node for node in G.nodes() 
                if G.in_degree(node) == 1 and G.out_degree(node) == 1 and not keep_nodes(G, node)
            ]
            
            if not nodes_to_remove:
                break # Если таких вершин нет, завершаем

            for node in nodes_to_remove:
                # Если узел уже был удален на предыдущей итерации этого же цикла, пропускаем
                if node not in G: 
                    continue

                # Получаем единственного предшественника и преемника
                # NetworkX гарантирует, что list(predecessors/successors) вернет один элемент,
                # если степень равна 1.
                u = list(G.predecessors(node))
                v = list(G.successors(node))

                if len(u) != 1 or len(v) != 1:
                    raise Exception('Этот эксепшен не должен никогда вызватся, но он вызвался и значит что то пошло не так')
                u = u[0]
                v = v[0]

                nodes_inside = sum((edge[-1]['nodes_inside'] for edge in G.edges(node, data = True)), [])
                    
                if u != v:
                    G.add_edge(u, v, nodes_inside = [node] + nodes_inside)

                G.remove_node(node)

        after = len(self.nodes)
        if not self.silence:
            print('removed', original - after, 'nodes.', f'Efficiency: {round(100*(1 - after/original), 1)}%')
    
    def save_as_nx_graph(self, path):
        nx.write_gml(self.nodes, path)

    def save_as_pyvis_html(self, path):
        net = Network(notebook = False, directed = True)
        net.from_nx(self.nodes)
        for node in net.nodes:
            node['label'] = node['type']
            if node['type'] == 'root':
                node['size'] = 30
            if node['id'] < 0:
                node['color'] = 'orange'
                node['size'] = 5

        net.show_buttons(filter_=['physics'])
        net.save_graph(path)

In [ ]:
neurons = pymaid.find_neurons()
all_skids = neurons.skeleton_id

for skid in tqdm(all_skids):
    try:
        neuron = pymaid.get_neuron(skid)
        structure = simplified_structure(neuron, silence=True)
        out_path = f'./neurons/{neuron.id}.gml'
        structure.save_as_nx_graph(out_path)

    except Exception as e:
        print(f"FAIL {skid}: {e}")

In [ ]:
# MULTICORE

# neurons = pymaid.find_neurons()
# all_skids = neurons.skeleton_id


# def process_neuron(skid):
#     try:
#         neuron = pymaid.get_neuron(skid)
#         structure = simplified_structure(neuron, silence=True)
#         out_path = f'./neurons/{neuron.id}.gml'
#         structure.save_as_nx_graph(out_path)
#         return f"OK {neuron.id}"
#     except Exception as e:
#         return f"FAIL {skid}: {e}"

# # Запуск в 72 потоках
# threads = 72
# with ThreadPoolExecutor(max_workers=threads) as executor:
#     futures = [executor.submit(process_neuron, skid) for skid in all_skids]

#     for future in tqdm(as_completed(futures), total=len(futures), desc="Processing neurons"):
#         print(future.result())

# Full Graph
Build the full graph out of all the neurons.

In [8]:
class composed_network:
    def __init__(self, path):
        self.paths = [os.path.abspath(f) for f in glob.glob(os.path.join(path, "*.gml"))]
        self.graphs = {}
        for path in tqdm(self.paths, desc="Loading neuron graphs"):
            self.graphs[path] = nx.read_gml(path)
            for node_id, attr_dict in self.graphs[path].nodes(True):
                filename = os.path.basename(path)
                attr_dict['owner'] = filename

        self.combined_graph = nx.compose_all(self.graphs.values())

    
    def save_as_nx_graph(self, path):
        nx.write_gml(self.combined_graph, path)

    
    def save_as_pyvis_html(self, path, colors:dict = None):
        net = Network(notebook=False, directed=True)
        net.from_nx(self.combined_graph)
        net.show_buttons(filter_=['physics'])
        for node in net.nodes:
            if node['type'] == 'root':
                node['size'] = 30
            if int(node['id']) < 0:
                node['shape'] = 'square'
                node['size'] = 5
                node['color'] = 'orange'
                if node['type'] not in ('Postsynaptic', 'Presynaptic'):
                    node['label'] = node['type']
                else:
                    node['label'] = None
            else:
                node['label'] = node['type']
                    
        if colors:
            for node in net.nodes:
                if int(node['id']) > 0:
                    if node['owner'] in colors:
                        node['color'] = colors[node['owner']]
        net.save_graph(path)

In [3]:
network = composed_network('./neurons')
network.save_as_nx_graph('./complete_graph.gml')

  0%|          | 0/5013 [00:00<?, ?it/s, Loading neuron graphs...]